# **6. Pipeline** **de** **procesamiento** **de** **texto** **en** **recetas**

En este módulo se define el pipeline que va a utilizarse para procesar las recetas a partir de los modelos NER y clasificador  entrenados.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # autoriza en el popup

Mounted at /content/drive


1. Carga del modelo clasificador de pasos y sus funciones asociadas (*_rm_acc*, *_build_norm_with_map*, *oraciones_con_offsets*, *_overlap*, *localizar_pasos_en_resumen*, *etiquetar_bi_por_oracion*, *construir_oraciones_dataset_bi*, *predecir_bi_por_oracion*, *reconstruir_pasos_desde_BI*)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

LOAD_DIR = "/content/drive/MyDrive/TFM/Classifier_model"
tokenizer_class = AutoTokenizer.from_pretrained(LOAD_DIR)
modelo_class     = AutoModelForSequenceClassification.from_pretrained(LOAD_DIR)

In [ ]:
import re, unicodedata, numpy as np, torch
from typing import List, Dict, Any, Tuple
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
def _rm_acc(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def _build_norm_with_map(text: str):
    """Normaliza (minúsculas, sin tildes, colapsa espacios) + mapa a índices del original."""
    norm_chars, idx_map = [], []
    last_space = False
    for i, ch in enumerate(text):
        base = _rm_acc(ch).lower()
        if base.isspace():
            if not last_space:
                norm_chars.append(" ")
                idx_map.append(i)
            last_space = True
        else:
            norm_chars.append(base)
            idx_map.append(i)
            last_space = False
    norm = " ".join("".join(norm_chars).strip().split())
    idx_map = idx_map[:len(norm)]
    return norm, idx_map

def oraciones_con_offsets(texto: str):
    """Divide por . ! ? …  y devuelve [(start,end,oración)] (end exclusivo)."""
    bounds, start = [], 0
    for m in re.finditer(r"[.!?…]+", texto):
        end = m.end()
        if end > start:
            bounds.append((start, end))
        nxt = end
        while nxt < len(texto) and texto[nxt].isspace():
            nxt += 1
        start = nxt
    if start < len(texto):
        bounds.append((start, len(texto)))
    return [(a,b,texto[a:b]) for a,b in bounds]

def _overlap(a1,b1,a2,b2):
    return max(0, min(b1,b2) - max(a1,a2))

def localizar_pasos_en_resumen(resumen: str, pasos: List[str]):
    """Encuentra pasos en el resumen con búsqueda robusta (normalizada)."""
    offs, cursor_norm = [], 0
    H, Hmap = _build_norm_with_map(resumen)
    for p in (pasos or []):
        p = (p or "").strip()
        if not p:
            continue
        N, _ = _build_norm_with_map(p)
        if not N:
            continue
        pos = H.find(N, cursor_norm)
        if pos == -1:
            pos = H.find(N)
            if pos == -1:
                continue
        try:
            s = Hmap[pos]
            e = Hmap[pos + len(N) - 1] + 1
            offs.append((s, e))
            cursor_norm = pos + len(N)
        except Exception:
            continue
    return offs

def etiquetar_bi_por_oracion(resumen: str, pasos: List[str]) -> List[str]:
    """
    Etiqueta BI por oración:
      - 'B' si empieza un paso (o cambia de paso),
      - 'I' si continúa el mismo paso.
    Si una oración no solapa con ningún paso, continúa el actual (o abre 'B' si es la primera).
    """
    sents = oraciones_con_offsets(resumen)
    steps = localizar_pasos_en_resumen(resumen, pasos)
    if not sents:
        return []
    asign = []
    for (sa, sb, _) in sents:
        best_k, best_ov = None, 0
        for k, (pa, pb) in enumerate(steps):
            ov = _overlap(sa, sb, pa, pb)
            if ov > best_ov:
                best_k, best_ov = k, ov
        asign.append(best_k if best_ov > 0 else None)

    labels, prev_step = [], None
    for i, k in enumerate(asign):
        if k is None:
            k = prev_step if prev_step is not None else 0
        labels.append("B" if (prev_step is None or k != prev_step) else "I")
        prev_step = k
    return labels

In [ ]:
def construir_oraciones_dataset_bi(recetas: List[Dict[str,Any]], usar_contexto: bool = True):
    filas = []
    for r in recetas:
        resumen = (r.get("resumen") or "").strip()
        pasos   = r.get("pasos") or []
        if not resumen:
            continue
        sents = oraciones_con_offsets(resumen)
        if not sents:
            continue
        labels = etiquetar_bi_por_oracion(resumen, pasos)
        if len(labels) != len(sents):
            labels = ["I"] * len(sents)  # fallback seguro

        for i, (_,_,sent) in enumerate(sents):
            row = {"text": sent, "label": labels[i]}
            if usar_contexto:
                row["prev"] = sents[i-1][2] if i>0 else ""
                row["next"] = sents[i+1][2] if i < len(sents)-1 else ""
            filas.append(row)
    return filas

In [ ]:
@torch.no_grad()
def predecir_bi_por_oracion(resumen: str, tokenizer, model, usar_contexto: bool = True):
    model.eval()
    sents = oraciones_con_offsets(resumen)
    if not sents:
        return []
    textos = []
    for i, (_,_,sent) in enumerate(sents):
        if usar_contexto:
            prev_txt = sents[i-1][2] if i>0 else ""
            next_txt = sents[i+1][2] if i < len(sents)-1 else ""
            sep = tokenizer.sep_token or "</s>"
            textos.append(f"{prev_txt.strip()} {sep} {sent.strip()} {sep} {next_txt.strip()}".strip())
        else:
            textos.append(sent)
    enc = tokenizer(textos, padding=True, truncation=True, max_length=256, return_tensors="pt")
    enc = {k: v.to(model.device) for k,v in enc.items()}
    logits = model(**enc).logits
    preds  = logits.argmax(-1).cpu().numpy().tolist()
    return [id2label[i] for i in preds]  # "B" o "I"

def reconstruir_pasos_desde_BI(resumen: str, etiquetas_bi: List[str]):
    sents = oraciones_con_offsets(resumen)
    assert len(sents) == len(etiquetas_bi)
    pasos_offsets, pasos_textos = [], []
    cur = None
    def cerrar():
        nonlocal cur
        if cur:
            a,b = cur
            pasos_offsets.append((a,b))
            pasos_textos.append(resumen[a:b])
        cur = None
    for (a,b,_), tag in zip(sents, etiquetas_bi):
        tag = (tag or "I").upper()
        if tag == "B" or cur is None:
            cerrar(); cur = [a,b]
        else:  # I
            cur[1] = b
    cerrar()
    return {"pasos": pasos_textos, "pasos_offsets": pasos_offsets}


In [ ]:
label_list = ["B","I"]
label2id   = {"B":0, "I":1}
id2label   = {0:"B", 1:"I"}

Ejemplo de salida al aplicar el modelo clasificador, con las entradas *'Etiquetas'* que clasifica cada frase según sea el inicio de un paso o la continuación del mismo, y la entrada *'Pasos'* que genera los pasos de la receta a partir de las etiquetas de las oraciones y su orden.

In [ ]:
texto = ("Pon en la olla ajo. Mezclalo con la cebolla. Despues añade el tomate")
y_bi = predecir_bi_por_oracion(texto, tokenizer_class, modelo_class, usar_contexto=True)
seg  = reconstruir_pasos_desde_BI(texto, y_bi)
print("Etiquetas:", y_bi)            # p.ej. ['B','I','B']
print("Pasos:", seg["pasos"])        # subcadenas EXACTAS del resumen
print("Offsets:", seg["pasos_offsets"])

Etiquetas: ['B', 'I', 'B']
Pasos: ['Pon en la olla ajo. Mezclalo con la cebolla.', 'Despues añade el tomate']
Offsets: [(0, 44), (45, 68)]


2. Carga del modelo NER de etiquetado de ingredientes.

In [ ]:
import torch
LOAD_DIR = "/content/drive/MyDrive/TFM/ner_model_2"
tokenizer_ner = AutoTokenizer.from_pretrained(LOAD_DIR, use_fast=True)  # <- clave
modelo_ner    = AutoModelForTokenClassification.from_pretrained(LOAD_DIR)

In [ ]:
from transformers import pipeline
import torch

clf = pipeline(
    "token-classification",
    model=modelo_ner,
    tokenizer=tokenizer_ner,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cpu


Diseño del algoritmo de obtención de ingredientes por paso. Se aplica el modelo de reconocimiento de ingredientes sobre cada paso de la receta, previamente procesada con el clasificador de pasos. Los ingredientes se incluyen en listas asociadas a cada paso, que forman una lista con la receta completa. A modo de ejemplo se aplica a la receta definida antes.

In [ ]:
labels_por_paso = []

for paso in seg.get("pasos", []):
    if isinstance(paso, str) and paso.strip():
        ents = clf(paso)
    else:
        ents = []

    labels = []
    for e in ents:
        lab = (e.get("entity_group") or e.get("entity") or "")
        lab = lab.lower().replace("_", " ").strip()
        if lab:
            labels.append(lab)

    labels_por_paso.append(labels)

seg["labels_por_paso"] = labels_por_paso

In [ ]:
seg["labels_por_paso"]

[['ajo', 'cebolla'], ['tomate']]

Integración del clasificador de pasos y el anotador de ingredientes mediante la función ***procesado_texto_receta***. El pipeline se basa en aplicar al conjunto del texto en crudo el clasificador para separarlos en pasos, y procesar paso a paso mediante el algoritmo anteriormente descrito obteniendo los ingredientes de cada paso.

In [ ]:
def procesado_texto_receta(receta):

# 2) Segmentación BI -> pasos
    y_bi = predecir_bi_por_oracion(texto, tokenizer_class, modelo_class, usar_contexto=True)
    seg  = reconstruir_pasos_desde_BI(texto, y_bi)

    # 3) NER por paso -> labels por paso
    labels_por_paso = []
    for paso in seg.get("pasos", []):
        if isinstance(paso, str) and paso.strip():
            ents = clf(paso)  # pipeline(token-classification)
        else:
            ents = []

        labels = []
        for e in ents:
            lab = (e.get("entity_group") or e.get("entity") or "")
            lab = lab.lower().replace("_", " ").strip()
            if lab:
                labels.append(lab)
        labels_por_paso.append(labels)

    # 4) Construir salida
    receta_procesada = {
        "pasos": seg.get("pasos", []),
        "ingredientes": labels_por_paso
    }
    return receta_procesada

Probamos la función de procesado de recetas con distintas variaciones para valorar cualitativamente su desempeño.

In [ ]:
texto = "Pon en la olla ajo. Mezclalo con la cebolla. Despues añade el tomate"
receta_procesada_1 = procesado_texto_receta(texto)
print(receta_procesada_1)

{'pasos': ['Pon en la olla ajo. Mezclalo con la cebolla.', 'Despues añade el tomate'], 'ingredientes': [['ajo', 'cebolla'], ['tomate']]}


In [ ]:
texto = 'Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños. Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados. Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'
receta_procesada_2 = procesado_texto_receta(texto)
print(receta_procesada_2)

{'pasos': ['Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños.', 'Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados.', 'Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'], 'ingredientes': [[], ['huevo', 'pollo'], ['aceite']]}


In [ ]:
texto = "Comenzamos haciendo la chistorra. La vamos a saltear en un sartén sin aceite ya que ella misma va a soltar mucha grasa.  Una vez cocida la retiramos y reservamos. Pelamos y partimos las patatas a lo largo. La freímos en abundante aceite de oliva. Ahora para hacer unos huevos de calidad, cogemos y cubrimos el fondo de una sartén con aceite de oliva. Cuando esté bien caliente, añadimos el huevo con un toque de sal por encima.  Con una espátula vamos echando por encima aceite para que se haga la yema un poco y listo. Sacamos el huevo y reservamos. Servimos las patatas con la chistorra y por encima el huevo frito y tenemos uno de los mejores platos de las abuelas."
receta_procesada_3 = procesado_texto_receta(texto)
print(receta_procesada_3)

{'pasos': ['Comenzamos haciendo la chistorra.', 'La vamos a saltear en un sartén sin aceite ya que ella misma va a soltar mucha grasa.  Una vez cocida la retiramos y reservamos.', 'Pelamos y partimos las patatas a lo largo.', 'La freímos en abundante aceite de oliva.', 'Ahora para hacer unos huevos de calidad, cogemos y cubrimos el fondo de una sartén con aceite de oliva.', 'Cuando esté bien caliente, añadimos el huevo con un toque de sal por encima.', 'Con una espátula vamos echando por encima aceite para que se haga la yema un poco y listo. Sacamos el huevo y reservamos.', 'Servimos las patatas con la chistorra y por encima el huevo frito y tenemos uno de los mejores platos de las abuelas.'], 'ingredientes': [[], [], ['patata'], ['aceite'], ['huevo', 'aceite'], ['huevo', 'sal'], ['huevo'], ['patata', 'huevo']]}


Receta Tortilla de patatas

In [ ]:
texto = "Pelamos las patatas y las cortamos en gajos finos. Picamos la cebolla en  trocitos . Mezclamos las patatas con la cebolla en la sarten y lo freímos. Una vez pochadada las patatas y la cebolla, lo mezclamos con el huevo batido. Por último ponemos la mezcla en la sarten"
receta_procesada_tortilla = procesado_texto_receta(texto)
print(receta_procesada_tortilla)

{'pasos': ['Pelamos las patatas y las cortamos en gajos finos.', 'Picamos la cebolla en  trocitos .', 'Mezclamos las patatas con la cebolla en la sarten y lo freímos.', 'Una vez pochadada las patatas y la cebolla, lo mezclamos con el huevo batido.', 'Por último ponemos la mezcla en la sarten'], 'ingredientes': [['patata'], ['cebolla'], ['patata', 'cebolla'], ['patata', 'cebolla', 'huevo'], []]}
